# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,243.656881,0.707966,1234,244.364847,242.948914,488.729694,485.897829
6,26.748009,-0.195656,1234,26.552353,26.943665,53.104705,53.887329
7,238.373117,0.300883,1234,238.674000,238.072233,477.348000,476.144466
8,619.051665,-0.391945,1234,618.659721,619.443610,1237.319442,1238.887220
9,383.137270,-0.253204,1234,382.884066,383.390475,765.768132,766.780949
10,689.200519,0.160913,1234,689.361432,689.039606,1378.722864,1378.079212
11,933.602917,0.653054,1234,934.255971,932.949863,1868.511942,1865.899725
12,443.561081,0.502582,1234,444.063663,443.058500,888.127326,886.117000
13,923.482273,0.860897,1234,924.343170,922.621376,1848.686339,1845.242752
14,232.132264,-0.353664,1234,231.778601,232.485928,463.557201,464.971855


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-244.364847,242.948914
6,-26.552353,26.943665
7,-238.674000,238.072233
8,-618.659721,619.443610
9,-382.884066,383.390475
10,-689.361432,689.039606
11,-934.255971,932.949863
12,-444.063663,443.058500
13,-924.343170,922.621376
14,-231.778601,232.485928


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-244.364847,242.948914
6,-26.552353,26.943665
7,-238.674000,238.072233
8,-618.659721,619.443610
9,-382.884066,383.390475
10,-689.361432,689.039606
11,-934.255971,932.949863
12,-444.063663,443.058500
13,-924.343170,922.621376
14,-231.778601,232.485928


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-244.36484708,  242.9489143 ],
       [ -26.55235272,   26.94366459],
       [-238.67399998,  238.07223311],
       [-618.65972086,  619.44360991],
       [-382.88406605,  383.3904747 ],
       [-689.36143191,  689.03960584],
       [-934.25597099,  932.94986252],
       [-444.06366295,  443.05849986],
       [-924.3431695 ,  922.62137588],
       [-231.7786006 ,  232.48592766]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,486.729694,-485.897829
6,51.104705,-53.887329
7,475.348000,-476.144466
8,1235.319442,-1238.887220
9,763.768132,-766.780949
10,1376.722864,-1378.079212
11,1866.511942,-1865.899725
12,886.127326,-886.117000
13,1846.686339,-1845.242752
14,461.557201,-464.971855


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
